In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

import json
import time
import copy
from time import sleep
from selenium import webdriver

In [40]:
url = 'https://www.ebay.com/urw/Google-Home-Mini-Smart-Speaker-with-Google-Assistant-Chalk-GA00210-US-/product-reviews/239112716'
url1 = 'https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667'
url2 = 'https://www.ebay.com/urw/Sony-PlayStation-4-500GB-Jet-Black-Console/product-reviews/167317351'
url3 = 'https://www.ebay.com/urw/Nintendo-Switch-HAC-001-01-32GB-Console-with-Gray-Joy-Con/product-reviews/25034355190?condition=new'

url4 = 'https://www.ebay.com/urw/Milwaukee-2407-20-12V-Cordless-Drill-Driver/product-reviews/28030363313'
url5 = 'https://www.ebay.com/urw/Makita-XPH12Z-18V-Lithium-Ion-Cordless-Hammer-Drill/product-reviews/809538545'
url6 = 'https://www.ebay.com/urw/DeWalt-DCD791B-20V-Max-XR-Li-Ion-Brushless-Compact-Drill-Tool-Only-/product-reviews/13031982751'

url7 = 'https://www.ebay.com/urw/Canon-EOS-5D-Mark-III-22-3MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/113297147'
url8 = 'https://www.ebay.com/urw/Nikon-D7500-20-9MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/237096362'
url9 = 'https://www.ebay.com/urw/Nikon-D750-24-3-MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/203954517'

r = requests.get(url1)
r.status_code

200

In [273]:
#soup.find_all('p','review-item-content rvw-wrap-spaces')[0].find('span','show-full-review').text
#soup.find_all('div','review-section-header')
#lst = [i.text for i in soup.find_all('div','reviews-divider')]

def parse_product_category(url1):
    '''
    Returns the product category for a given product
    Input: URL of the product page to be parsed
    Output: String with the product category name
    '''
    product_category = 'Error'
    r1 = requests.get(url1)
    if r1.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        
    soup1 = BeautifulSoup(r1.text,'html.parser')
    for i in range(len(soup1.find_all('a','thrd'))):
        if type(soup1.find_all('a','thrd')[i].find('span')) is not type(None):
            return soup1.find_all('a','thrd')[i].find('span').text
    return product_category
  
def parse_product_type(url1):    
    '''
    Returns the product type for a given product
    Input: URL of the product page to be parsed
    Output: String with the product type name
    '''
    product_type = 'Error'
    r2 = requests.get(url1)
    if r2.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        
    soup2 = BeautifulSoup(r2.text,'html.parser')
    
    for i in range(len(soup2.find_all('a','scnd'))):
        if type(soup2.find_all('a','scnd')[i].find('span')) is not type(None):
            return soup2.find_all('a','scnd')[i].find('span').text
    return product_type

def text_word_count(txt):    
    '''
    Returns the word count for a given test
    Input: URL of the product page to be parsed
    Output: String with the product type name
    '''
    word_count = len([elem.strip("(),.!?;:[]{}\|*%$#@/`~") 
                      for elem in txt.split(' ') 
                      if elem.strip("(),.!?;:[]{}\|*%$#@/`~")!=''])
    return word_count    

#url10 = 'https://www.ebay.com/itm/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/154127032955?epid=175299667&hash=item23e2afe27b:g:bjYAAOSwHXxfflK~'
#print(parse_product_category(url10))


#Store all reviews on a given page
def parse_review_page(review_url, product_url, df):
    '''
    Inserts rows into Mongo DB and pandas df 
    Input: URL of the review page and product page to be parsed, pandas Dataframe df
    Output: Dictionary of category lists as values with category names as keys 
    '''
    r = requests.get(review_url)
    if r.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        return False
    
    soup = BeautifulSoup(r.text,'html.parser')
    
    for i in range(len(soup.find_all('p','review-item-content rvw-wrap-spaces'))):
        text = soup.find_all('p','review-item-content rvw-wrap-spaces')[i].find('span','show-full-review').text
        review_text = text
        word_count = text_word_count(text)
        star_rating = float(soup.find_all('div','ebay-star-rating')[i+1].find('span','star-rating')["aria-label"].split(' ')[0])
        avg_rating = float(soup.find_all('div','ebay-star-rating')[0].find('span','star-rating')["aria-label"].split(' ')[0])
        product_name = soup.find('div','row').find('a','page-title')['title']
        product_type = parse_product_type(product_url)
        category = parse_product_category(product_url)
        new_row = {'category':category,'product_type':product_type,'product_name':product_name,'avg_rating':avg_rating,
                   'review_text':review_text,'word_count':word_count,'star_rating':star_rating}
        df.append(new_row,ignore_index=True)
        #AA: Insert into MongoDB I
    return True
'''    
for i in range(len(soup.find_all('p','review-item-content rvw-wrap-spaces'))):
    text = soup.find_all('p','review-item-content rvw-wrap-spaces')[i].find('span','show-full-review').text
    review_text.append(text)
    word_count.append(text_word_count(text))
    star_rating.append(float(soup.find_all('div','ebay-star-rating')[i+1].find('span','star-rating')["aria-label"].split(' ')[0]))
    avg_rating.append(float(soup.find_all('div','ebay-star-rating')[0].find('span','star-rating')["aria-label"].split(' ')[0]))
    product_name.append(soup.find('div','row').find('a','page-title')['title'])
    product_type.append('gaming console')
    category.append('gaming')
    price.append(float('500'))
    
'''
#Create an empty DataFrame with all 
def create_empty_dataframe():
    category = ['null']
    product_type = ['null']
    product_name = ['null']
    avg_rating = ['null']
    review_text = ['null']
    star_rating = ['null']
    word_count = ['null'] 
    df = pd.DataFrame({'category':category,'product_type':product_type,'product_name':product_name,'avg_rating':avg_rating,
                       'review_text':review_text,'word_count':word_count,'star_rating':star_rating})    
    return df


product_url = 'https://www.ebay.com/itm/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/154127032955?epid=175299667&hash=item23e2afe27b:g:bjYAAOSwHXxfflK~'
review_url = url1
df = create_empty_dataframe()

if parse_review_page(review_url, product_url, df):
    print('in here')
    df.head(10)


in here


In [280]:
df.append({'category':'l','product_type':'l','product_name':'l','avg_rating':'l',
            'review_text':'l','word_count':'l','star_rating':'l'},ignore_index=True)
df.head()

,category,product_type,product_name,avg_rating,review_text,word_count,star_rating
0,null,null,null,null,null,null,null
